In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

import mpl_interactions.ipyplot as iplt
import matplotlib.pyplot as plt
from bsccm import BSCCM
from skimage import transform
import zarr
import numpy as np
from tqdm import tqdm
from numcodecs import Blosc
from skimage import transform

data_root = '/home/hpinkard_waller/2tb_ssd/BSCCM-coherent/'
export_root = '/home/hpinkard_waller/2tb_ssd/BSCCM-coherent-tiny/'
num_cells = 100
COHERENT = True

# data_root = '/home/hpinkard_waller/2tb_ssd/BSCCM/'
# export_root = '/home/hpinkard_waller/2tb_ssd/BSCCM-tiny/'
# num_cells = 1000
# COHERENT = False

bsccm = BSCCM(data_root)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Opening BSCCM (this may take a few seconds)...
BSCCM Opened


In [11]:
new_file = zarr.open(export_root + 'BSCCM_images.zarr', mode='w')

random_subset = np.random.choice(np.arange(bsccm.index_dataframe.global_index.size), size=num_cells, replace=False)
random_subset = np.sort(random_subset)

#resave new surface markers
bsccm.index_dataframe.iloc[random_subset].to_csv(export_root + 'BSCCM_index.csv', index=False)
bsccm.surface_marker_dataframe.iloc[random_subset].to_csv(export_root + 'BSCCM_surface_markers.csv', index=False)


new_dframe = bsccm.index_dataframe.iloc[random_subset]
for i in tqdm(range(new_dframe.global_index.size)):

    entry = new_dframe.iloc[i]  
    do_histology = not COHERENT and entry['has_matched_histology_cell']


    base_path = entry['data_path']
    fluor_dest = base_path + '/fluor/cell_{}'.format(entry.global_index)
    lf_dest = base_path + '/led_array/cell_{}'.format(entry.global_index)
    dpc_dest = base_path + '/dpc/cell_{}'.format(entry.global_index)
    hist_dest = base_path + '/histology/cell_{}'.format(entry.global_index)

    fluor_data = np.array([bsccm.read_image(entry.global_index, contrast_type='fluor', channel=c) for c in bsccm.fluor_channel_names])
    led_array_data = np.array([bsccm.read_image(entry.global_index, contrast_type='led_array', channel=c) for c in bsccm.led_array_channel_names])
    dpc_data = bsccm.read_image(entry.global_index, contrast_type='dpc')
    if do_histology:
        histology_data = bsccm.read_image(entry.global_index, contrast_type='histology', convert_histology_rgb32=False)

    new_file.create_dataset(name=fluor_dest, compressor=Blosc(cname='zstd', clevel=9, shuffle=Blosc.SHUFFLE),
                            data=fluor_data, chunks=(1, fluor_data.shape[-2], fluor_data.shape[-1]))
    new_file.create_dataset(name=lf_dest, compressor=Blosc(cname='zstd', clevel=9, shuffle=Blosc.SHUFFLE),
                            data=led_array_data, chunks=(1, led_array_data.shape[-2], led_array_data.shape[-1]))
    new_file.create_dataset(name=dpc_dest, compressor=Blosc(cname='zstd', clevel=9, shuffle=Blosc.SHUFFLE),
                            data=dpc_data, chunks=(dpc_data.shape[-2], dpc_data.shape[-1]))


    if do_histology:
        new_file.create_dataset(name=hist_dest, compressor=Blosc(cname='zstd', clevel=9, shuffle=Blosc.SHUFFLE),
                            data=histology_data, chunks=None)
        

100%|██████████| 100/100 [03:39<00:00,  2.19s/it]
